In [ ]:
from pyspark.sql import SparkSession
import asyncio 
import os

from azure_databricks.common.enums.etl_layers import ETLLayer
from azure_databricks.common.enums.env import Env
from azure_databricks.common.enums.domain_source import DomainSource # Nadal potrzebne, jeśli chcesz filtrować domeny
from azure_databricks.common.orchestrators.main_orchestrator import MainETLOrchestrator

import azure_databricks.common.orchestrators.register_all_domain_orchestrators 
import azure_databricks.common.transformers.register_all


spark = SparkSession.builder.appName("LayerETLPipeline").getOrCreate()
dbutils = get_ipython().user_ns["dbutils"]

ENVIRONMENT = Env.DEV
TARGET_ETL_LAYER = ETLLayer.BRONZE # <--- KLUCZOWY PARAMETR: Którą warstwę chcesz zbudować?

main_orchestrator = MainETLOrchestrator(
    spark=spark,
    dbutils_obj=dbutils,
    env=ENVIRONMENT
)

await main_orchestrator.execute_etl_layer_pipeline(target_etl_layer=TARGET_ETL_LAYER)

catalog_name = main_orchestrator.config.get_unity_catalog_name()
bronze_schema_name = ETLLayer.BRONZE.value
who_countries_bronze_table_name = f"{catalog_name}.{bronze_schema_name}.who_countries"

try:
    df_who_countries_bronze = spark.table(who_countries_bronze_table_name)
    df_who_countries_bronze.display()
except Exception as e:
    raise